<a href="https://colab.research.google.com/github/nachiiiket/OptiexIOT/blob/main/ScriptTester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random

# Input and Output files
input_file = "/content/test_sensor_data_1M.csv"
output_file = "test_sensor_data_1M_with_faulty.csv"

# Load the data
df = pd.read_csv(input_file)

# Total rows
n = len(df)

# Process in blocks of 10,000 rows
block_size = 10000
max_faulty_per_block = 100

for start in range(0, n, block_size):
    end = min(start + block_size, n)
    block_indices = list(range(start, end))

    # Randomly choose how many faulty values to inject (up to 100)
    num_faulty = random.randint(1, max_faulty_per_block)

    # Pick random positions within the block
    faulty_positions = random.sample(block_indices, num_faulty)

    for pos in faulty_positions:
        # Randomly decide if it's single zero or consecutive zeros
        if random.random() < 0.3:  # 30% chance to insert consecutive zeros
            run_length = random.randint(2, 5)  # 2 to 5 consecutive zeros
            for j in range(pos, min(pos + run_length, end)):
                df.at[j, "Value"] = 0
        else:
            df.at[pos, "Value"] = 0

# Save new CSV
df.to_csv(output_file, index=False)

print(f"✅ New dataset with faulty values saved to {output_file}")


In [2]:
import pandas as pd
import random

# Input and Output files
input_file = "test_sensor_data_1M.csv"
output_faulty = "test_sensor_data_1M_with_faulty.csv"
output_expected = "expected_output_1M.csv"

# Load data
df = pd.read_csv(input_file)

n = len(df)
block_size = 10000
max_faulty_per_block = 100

# --- STEP 1: Insert random faulty zeros ---
for start in range(0, n, block_size):
    end = min(start + block_size, n)
    block_indices = list(range(start, end))
    num_faulty = random.randint(1, max_faulty_per_block)
    faulty_positions = random.sample(block_indices, num_faulty)

    for pos in faulty_positions:
        if random.random() < 0.45:  # 30% chance to insert consecutive zeros
            run_length = random.randint(2, 5)
            for j in range(pos, min(pos + run_length, end)):
                df.at[j, "Value"] = 0
        else:
            df.at[pos, "Value"] = 0

# Save dataset with faulty values
df.to_csv(output_faulty, index=False)

# --- STEP 2: Generate Expected Output ---
df_expected = df.copy()
df_expected["Condition"] = "Normal"

for i in range(1, len(df_expected)):
    prev_val = df_expected.at[i-1, "Value"]
    curr_val = df_expected.at[i, "Value"]

    # Faulty Sensor → Zero values
    if curr_val == 0:
        df_expected.at[i, "Condition"] = "Faulty Sensor"
        continue

    # Reset Condition → Previous drop to very low then start increasing/stabilizing
    if prev_val > curr_val and i+1 < len(df_expected):
        next_val = df_expected.at[i+1, "Value"]
        if next_val >= curr_val:
            df_expected.at[i, "Condition"] = "Reset"
            continue

    # Negative Difference → Drop but not a reset
    if curr_val < prev_val:
        df_expected.at[i, "Condition"] = "Negative Difference"

# Save expected output
df_expected.to_csv(output_expected, index=False)

print(f"✅ Created {output_faulty} and {output_expected}")


✅ Created test_sensor_data_1M_with_faulty.csv and expected_output_1M.csv
